# Data Constructor

This notebook will take in a csv file containing the raw data straight from the [AUDL Stats](https://theaudl.com/league/stats) page. It takes this data and transformes it into usable training data for my machine learning model. Below is a quick summary of the tasks implemented in this notebook:

1. Load the raw data
2. Create dictionaries for quick acess to team names and abbreviations
3. Create a new DataFrame called `calulated_df`
4. populate `calculated_df` with identifying data (team name, opponent, date) for every recorded game 
5. Write several functions for calculating useful statistics
6. Populate `calculated_df` with over 50 new statistics for each game 
7. Export `calculated_df` as **AUDL_pd.csv**
8. Export the cleaned up raw data as **raw_improved_pd.csv**

#### Load Datasets and imports

In [4]:
import pandas as pd

### Load the csv files

raw_df: raw data directly from AUDL site

averages_df: containing team averages across a two year time span (2022-23) for various stats

In [5]:
raw_df = pd.read_csv(
    "RawData.csv",
    header = 0)

averages_df = pd.read_csv(
    "Team Averages.csv",
    header = 0)

# Dropping the last row (contains nothings)
# Removing duplicate columns and standardizing thier names

raw_df = raw_df.drop(columns=['Completion %',	'Huck %',	'Hold %',	'OLC %',	'Break %',	'DLC %',	'RZC %'])
raw_df = raw_df.rename(columns={'Huck %.1': 'Huck%', 'Hold %.1': "Hold%", 'Break %.1':'Break%', 'DLC %.1': 'DLC%', 'RZC %.1':'RZC%'})

raw_df.head()

,Team,Date,Score,Turnovers,Blocks,Completion%,Huck%,Hold%,OLC%,Break%,DLC%,RZC%,Away Score,Home Score,Points/Total
0,AlleyCats,6/4/2021,DET 21 - 30 IND,14,12,0.92,0.76,0.86,0.73,0.34,0.61,0.93,21,30,0.588
1,Breeze,6/4/2021,DC 18 - 19 NY,9,7,0.97,1.00,0.74,0.67,0.20,0.57,0.81,18,19,0.486
2,Cannons,6/4/2021,TB 16 - 21 PHI,25,10,0.89,0.44,0.48,0.35,0.22,0.44,0.69,16,21,0.432
3,Empire,6/4/2021,DC 18 - 19 NY,10,5,0.95,0.71,0.75,0.65,0.21,0.67,0.92,18,19,0.514
4,Mechanix,6/4/2021,DET 21 - 30 IND,23,8,0.93,0.55,0.63,0.51,0.05,0.17,0.82,21,30,0.412


### Dictionaries

Some information is used very frequently and makes sense to be put into a dictionary. For instance, the `team_dict` dictionary maps abbreviations to thier team name **(SLC, Shred)**. `abb_dict` does the inverse. 

`avg_dict` is simply the dictionary representation of `averages_df`. The team abbreviation maps to a list of statistics averages for that team

In [6]:
# Creates Dictionary of Team names and their abbreviations
num_teams = 25
team_dict = pd.Series(averages_df['Team Abb'][0:26].values,index=averages_df['Team Name'][0:num_teams+1]).to_dict()
abb_dict = pd.Series(averages_df['Team Name'].values,index=averages_df['Team Abb']).to_dict()

# Creates dictionary for averages
avg_dict = averages_df.set_index('Team Abb').T.to_dict('list')

### Create New Dataframe

Creates the `calculated_df` dataframe that will be added to for the rest of the notebook. 

In [7]:
# Creates new Dataframe, and adds Team names for all the games
calculated_df = pd.DataFrame()

calculated_df.index = raw_df.index
calculated_df['Team Name'] = raw_df['Team']

calculated_df.head()

,Team Name
0,AlleyCats
1,Breeze
2,Cannons
3,Empire
4,Mechanix


### Identifying Information

The next cells will add the first **9** columns of `calculated_df`. These include basic identifying information such as team names, opponent names, game location, date, and the number of games played so far in the season. 

The code comments explain more in depth what is happening in each cell. 



In [8]:
# Adds Home Team Abbreviations

# A dictionary of teams that no longer exist, and the final year they palyed
depreciated_teams = {"Flyers": ("RAL", 2021), "Spiders":("SJ" ,2021), "Sol":("AUS", 2021)}

def add_abb(column_name, df, raw):
    'Returns list of team abbreviations'
    result = []
    for i in range(df.shape[0]):
        team = df.loc[i,column_name]
        if(team in depreciated_teams.keys()):
            year = int(raw.loc[i, 'Date'][-4:])
            
            if(depreciated_teams[team][1] >= year):
                result.append(depreciated_teams[team][0])
                continue

        result.append(team_dict[team])
    return result

# Creating the Abbreviation column for the subject team
calculated_df['Abb'] = add_abb('Team Name',calculated_df, raw_df)


In [9]:
# Add Opponent Abbreviations
opp_result = []
for i in range(raw_df.shape[0]):       
        #abb_size = len(calculated_df['Abb'][i])
        if(raw_df['Score'][i][:3].strip() == calculated_df['Abb'][i]):
                opp = raw_df['Score'][i][-3:]
                opp = opp.strip()
                opp_result.append(opp)
        else:
                opp = raw_df['Score'][i][:3]
                opp = opp.strip()
                opp_result.append(opp)


# Creating the Abbreviation column for the opposing team
calculated_df['Opponent Abb'] = opp_result

In [10]:
# Adds Opposing Team names
opp_team_result = []
for ab in calculated_df['Opponent Abb']:
    opp_team_result.append(abb_dict[ab])


# Creates column for oppising team names
calculated_df['Opposing Team Name'] = opp_team_result
# Reorders the dataframe slightly by moving the column backwards
calculated_df.insert(2, "Opposing Team Name", calculated_df.pop('Opposing Team Name'))

In [11]:
# Adds the Date
calculated_df['Date'] = raw_df['Date']
calculated_df['Date']= pd.to_datetime(calculated_df['Date'])

In [12]:
# Adds Home or Away Game
home_result = []
for i in range(raw_df.shape[0]): 
    if(raw_df['Score'][i][0:len(calculated_df['Abb'][i])] == calculated_df['Abb'][i]):
        home_result.append(0)
    else:
        home_result.append(1)

calculated_df['Home Game'] = home_result


In [13]:
# Adds game number in Season
# Defines same_season function

def same_season(i):
    'Creates a mask of all games within the same season of a particular row'
    end_date = pd.to_datetime(calculated_df['Date'][i])
    start_date = pd.to_datetime(calculated_df['Date'][i].year, format='%Y')

    lt_date = calculated_df['Date'] < end_date
    gt_season = calculated_df['Date'] >= start_date
    
    return (lt_date & gt_season)
    

def game_number(team_column):
    'Calculates how many games a team has palyed in the season'
    game_number_result = []

    for i in range(raw_df.shape[0]): 
        ss = same_season(i)
        name_match = calculated_df['Team Name'] == calculated_df[team_column][i]

        # sums up all the games that meet both criteria, plus one for the current game
        counts = (ss & name_match).sum() + 1

        game_number_result.append(counts)
    return game_number_result


# How many games the subject team has palyed
calculated_df['Game Number'] = game_number('Team Name')
# How many games the opponent has palyed
calculated_df['Oppenent Game Number'] = game_number('Opposing Team Name')


In [14]:
# Game Against Opponent

games_vs_oppenent_result = []
for i in range(calculated_df.shape[0]):
    same_matchup = (calculated_df['Team Name'] == calculated_df['Team Name'][i]) & (calculated_df['Opposing Team Name'] == calculated_df['Opposing Team Name'][i])
    ss = same_season(i)
    games_vs_oppenent_result.append((same_matchup & ss).sum() + 1)

# Number of Games the two teams have palyed against eachother
calculated_df['Game Against Opponent']  = games_vs_oppenent_result
calculated_df.head()


,Team Name,Abb,Opposing Team Name,Opponent Abb,Date,Home Game,Game Number,Oppenent Game Number,Game Against Opponent
0,AlleyCats,IND,Mechanix,DET,2021-06-04,1,1,1,1
1,Breeze,DC,Empire,NY,2021-06-04,0,1,1,1
2,Cannons,TB,Phoenix,PHI,2021-06-04,0,1,1,1
3,Empire,NY,Breeze,DC,2021-06-04,1,1,1,1
4,Mechanix,DET,AlleyCats,IND,2021-06-04,0,1,1,1


### Game Stats

The next cells will use a single function to calculate important statistics. For each statistic, for example **Completion%**, there will be 6 slightly different averages taken:

1. Average Completion% over the past 5 games for the subject team
2. Average Completion% over the season for the subject team

3. Average Completion% over the past 5 games for the opposing team
4. Average Completion% over the season for the opposing team

5. Average Completion% against the opponent
6. Average Completion% of the opponent against the subject team


As stated, this will be repeated for other metrics such as **RZC%**, **Hold%**, **Break%**, and many others. If you do not know what a specific statistic means, visit the AUDL stats webiste linked at the top of this page. 

In [15]:
# stat_pct Formula Define
# The main function used to calculate averages based on several criteria

# Params: 
#       team_name_column: whether this is the subject or opponent
#       ranger: n games in the past, -1 if you want entire season
#       average_column: which column in raw_df to average
#       stat_num: index of stat column in avg_dict dictionary for baseline statistics
#       other_name_column: Optional,used if you wish to only take the average against a certain opponent

def stat_pct(team_name_column, ranger, average_column, stat_num, other_name_column = ''):
    'Calculates the completion precent averages over a period of time, or against a particular opponent'
    completion_pct_result = []
    for i in range(calculated_df.shape[0]):
        # Only games in current season
        ss = same_season(i)
        # Only games that the subject team has palyed
        name_match = calculated_df['Team Name'] == calculated_df[team_name_column][i]
        # Only games in the past
        games_lt = (calculated_df['Game Number'] < calculated_df['Game Number'][i])
        games_gt = 1

        if(ranger != -1):
            # If a range is specified, use games within that range
            games_gt = (calculated_df['Game Number'] >= (calculated_df['Game Number'][i] - ranger) )

        if(other_name_column != ''):
            # If the optional argument is specified, create a mask to select only the games where the two teams play against eachother
            name_match = name_match & (calculated_df['Opposing Team Name'] == calculated_df[other_name_column][i])
        
        # Final mask
        mask = ss & name_match & games_lt & games_gt

        # If no games meet this criteria, take an average from the avg_dict dictionary
        if(mask.sum() == 0):
            abb: str
            if(team_name_column == 'Team Name'):
                abb = calculated_df['Abb'][i]
            else:
                abb = calculated_df['Opponent Abb'][i]
            completion_pct_result.append(round(avg_dict[abb][stat_num], 3))
            continue

        average = raw_df[average_column][mask].mean()
        completion_pct_result.append(round(average, 3))

    return completion_pct_result

In [16]:
# Completion %

avg_index = 2 # Used to find the right statistic in the avg_dict dictionary (column index of team_averages dataframe)
stat = "Completion%"
avgerage_column = 'Completion%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [17]:
# Huck %

avg_index = 3
stat = "Huck%"
avgerage_column = 'Huck%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')


In [18]:
# Hold %

avg_index = 4
stat = "Hold%"
avgerage_column = 'Hold%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [19]:
# OLC %

avg_index = 5
stat = "OLC%"
avgerage_column = 'OLC%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [20]:
# Break %

avg_index = 6
stat = "Break%"
avgerage_column = 'Break%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [21]:
# DLC %

avg_index = 7
stat = "DLC%"
avgerage_column = 'DLC%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [22]:
# RZC %

avg_index = 8
stat = "RZC%"
avgerage_column = 'RZC%'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [23]:
# Turnovers

avg_index = 9
stat = "Turnovers"
avgerage_column = 'Turnovers'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [24]:
#  Blocks

avg_index = 10
stat = "Blocks"
avgerage_column = 'Blocks'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [25]:
# Points / Total

avg_index = 11
stat = "Points/Total"
avgerage_column = 'Points/Total'

calculated_df[f'{stat} (Last 5 Games)'] = stat_pct('Team Name', 5, avgerage_column , avg_index)
calculated_df[f'{stat} (Season)'] = stat_pct('Team Name', -1, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Last 5 Games)'] = stat_pct('Opposing Team Name', 5, avgerage_column ,avg_index)
calculated_df[f'{stat} Opponent (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index)
calculated_df[f'{stat} Against Opponent (Season)'] = stat_pct('Team Name', -1,avgerage_column , avg_index,'Opposing Team Name')
calculated_df[f'{stat} Opponent Against Home (Season)'] = stat_pct('Opposing Team Name',-1,avgerage_column , avg_index,'Team Name')

In [26]:
# Win or Loss (1 or 0)

calculated_df["Result"] = (raw_df['Points/Total'] > .5).astype(int)

### Export to .csv

Export the final dataframes to csv. As more games and seasons of the AUDL are palyed, this notebook can be run to make a larger and more comprehensive tables.

In [27]:
calculated_df.to_csv('AUDL_pd.csv', index=False)
raw_df.to_csv('raw_improved_pd.csv', index=False)

In [28]:
calculated_df.head()

,Team Name,Abb,Opposing Team Name,Opponent Abb,Date,Home Game,Game Number,Oppenent Game Number,Game Against Opponent,Completion% (Last 5 Games),...,Blocks Opponent (Season),Blocks Against Opponent (Season),Blocks Opponent Against Home (Season),Points/Total (Last 5 Games),Points/Total (Season),Points/Total Opponent (Last 5 Games),Points/Total Opponent (Season),Points/Total Against Opponent (Season),Points/Total Opponent Against Home (Season),Result
0,AlleyCats,IND,Mechanix,DET,2021-06-04,1,1,1,1,0.948,...,7.2,8.7,7.2,0.499,0.499,0.388,0.388,0.499,0.388,1
1,Breeze,DC,Empire,NY,2021-06-04,0,1,1,1,0.950,...,10.6,10.6,10.6,0.551,0.551,0.567,0.567,0.551,0.567,0
2,Cannons,TB,Phoenix,PHI,2021-06-04,0,1,1,1,0.895,...,9.4,8.4,9.4,0.408,0.408,0.489,0.489,0.408,0.489,0
3,Empire,NY,Breeze,DC,2021-06-04,1,1,1,1,0.948,...,10.6,10.6,10.6,0.567,0.567,0.551,0.551,0.567,0.551,1
4,Mechanix,DET,AlleyCats,IND,2021-06-04,0,1,1,1,0.908,...,8.7,7.2,8.7,0.388,0.388,0.499,0.499,0.388,0.499,0
